# Peak fitting of selected ROIs in diffraction data
Select one or more regions of interest and fit a simple gaussian peak across all frames

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import DanMAX as DM
style = DM.darkMode(style_dic={'size':'large'})

### Import data

In [ ]:
# Insert path for the .h5 file - TIP: Use tap for auto-complete
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/scan-XXXX.h5'
fname = DM.findScan()

# get the azimuthally integrated filename from master file name
aname = DM.getAzintFname(fname)

# read the integrated data
data = DM.getAzintData(aname)
if type(data['q']) != type(None):
    x = data['q']
    Q = True
else:
    x = data['tth']
    Q = False

# read common meta data from the master file
meta = DM.getMetaData(fname)

# reduce time-resolution to speed up initial analysis
rf = 1
start = None 
end =  None
data = DM.reduceDic(data,reduction_factor=rf,start=start,end=end)
meta = DM.reduceDic(meta,reduction_factor=rf,start=start,end=end)

I = data['I']

#meta_dic = DM.getMetaDic(fname) # extended meta data dictionary
t = meta['time'] # relative time stamp in seconds
T = meta['temp'] # temperature in Kelvin (if available, otherwise None)
I0 = meta['I0']  # relative incident beam intensity "I zero"
E = meta['energy'] # X-ray energy in keV

print(f'Effective time-resolution: {np.mean(np.diff(t)):.2f} s')

# normalize the integrated intensity to I0
I = (I.T/I0).T

### Select regions of interest
Defined in the relevant scattering unit

In [ ]:
# define the approximate region of interest in scattering units
#         label  :   roi               number of sub-peaks      
peaks = {'peak_1':{'roi' : [6.8,7.5]  ,'num_of_peaks' : 1},
         'peak_2':{'roi' : [9.8,10.4] ,'num_of_peaks' : 2},
        }

# plot the region of interest for the average pattern
plt.figure()
plt.title(DM.getScan_id(fname))
plt.plot(x,np.mean(I,axis=0),'-',c='grey',label='average pattern')
# loop through all peaks
for peak in peaks:
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    plt.plot(x[roi],np.mean(I,axis=0)[roi],'.',label=peak)
plt.ylabel('I [a.u.]')
if Q:
    plt.xlabel(r'Q [$\AA^{-1}$]')
else:
    plt.xlabel(r'2$\theta$ [$\deg$]')
plt.legend()

### Perform single-peak fitting

In [ ]:
fit_linear_background = True
# loop through all peaks
for peak in peaks:
    # create region of interest mask
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    
    amplitude, position, FWHM, background, I_calc = [],[],[],[],[]
    peaks[peak]['integral'] = np.full(I.shape[0],np.nan)
    for i,y in enumerate(I):
        if fit_linear_background:
            bgr_lin = DM.fitting.linearBackground(y[roi])
            bgr_lin += np.min(y[roi]-bgr_lin)
        peaks[peak]['integral'][i] = np.array(np.trapz(y[roi]-bgr_lin,x[roi]))
        amp,pos,fwhm,bgr,y_calc = DM.fitting.singlePeakFit(x[roi],y[roi]-bgr_lin)
        if np.isnan(amp):
            print(f'The fit of pattern {i+1} did not converge!' )
        amplitude.append(amp)
        position.append(pos)
        FWHM.append(fwhm)
        background.append(bgr+np.mean(bgr_lin))
        I_calc.append(y_calc+bgr_lin+bgr)
    # convert to numpy arrays and save results in the peaks dictionary
    peaks[peak]['amplitude'] = np.array(amplitude)
    peaks[peak]['position'] = np.array(position)
    peaks[peak]['FWHM'] = np.array(FWHM)
    peaks[peak]['background'] = np.array(background)
    peaks[peak]['I_calc'] = np.array(I_calc)
    #peaks[peak]['integral'] = np.array(np.trapz(I[:,roi],x[roi],axis=1))

# compare the average observed peak to the average fitted peak
global_roi = min([min(peaks[peak]['roi']) for peak in peaks]), max([max(peaks[peak]['roi']) for peak in peaks])
roi = (x>global_roi[0]) & (x<global_roi[1])

plt.figure()
plt.title(DM.getScan_id(fname))
plt.plot(x[roi],np.mean(I,axis=0)[roi],'.',c='grey',label='average observed peak')
for peak in peaks:
    # create region of interest mask
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    I_calc = peaks[peak]['I_calc']
    plt.plot(x[roi],np.nanmean(I_calc,axis=0),label='average fitted peak')
    
plt.ylabel('I [a.u.]')
if Q:
    plt.xlabel(r'Q [$\AA^{-1}$]')
else:
    plt.xlabel(r'2$\theta$ [$\deg$]')
plt.legend()

### Plot single-peak fitting results

In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True)
fig.suptitle(DM.getScan_id(fname))
axs = axs.flatten()

keys = ['amplitude', 'position', 'FWHM', 'background','integral']
for peak in peaks:
    for i,key in enumerate(keys):
        axs[i].plot(t,peaks[peak][key],label=peak)
        axs[i].set_xlabel('Time (s)')
        axs[i].set_title(key)
# plot temperature if available
if type(T) != type(None):
    axs[-1].plot(t,T, label='Temperature')
    axs[-1].set_title('temperature')
else:
    axs[-1].plot(t,I0, label='I0')
    axs[-1].set_title('I0')

axs[-1].set_xlabel('Time (s)')
axs[2].legend()
fig.tight_layout()

### Perform multi-peak fitting

In [ ]:
fit_linear_background = True

peak_multi = {}
bgr_lin = 0.

# loop through all peaks
for peak in peaks:
    # create region of interest mask
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    peaks[peak]['I_calc'] = np.full(I[:,roi].shape,np.nan)
    peaks[peak]['background'] = np.full(I.shape[0],np.nan)
    for i,y in enumerate(I):
        if fit_linear_background:
            bgr_lin = DM.fitting.linearBackground(y[roi])
            bgr_lin += np.min(y[roi]-bgr_lin)
        if i<1:
            sub_peaks,bgr = DM.fitting.multiPeakFit(x[roi],
                                                    y[roi]-bgr_lin,
                                                    peaks[peak]['num_of_peaks'])

            # initialize results dictionary
            for sub in sub_peaks:
                peak_multi[f'{peak}_{sub}'] = {'roi': peaks[peak]['roi'],
                                               'amplitude':[],
                                               'position':[],
                                               'FWHM':[],
                                               'integral':[],
                                               'y_calc':[],
                                        }
        
        else:
            sub_peaks,bgr = DM.fitting.multiPeakFit(x[roi],
                                                    y[roi]-bgr_lin,
                                                    peaks[peak]['num_of_peaks'],
                                                    sub_peaks,
                                                    verbose=False,
                                                   )
        peaks[peak]['I_calc'][i] = DM.fitting.multiGauss(x[roi],sub_peaks,bgr)+bgr_lin
        peaks[peak]['background'][i] = bgr+np.mean(bgr_lin)
        # unpack sub peak parameter results
        #sub_peaks = {'peak_0':[a,x0,sigma],
        #              ...
        #             'peak_n':[a,x0,sigma]}
        for i,sub in enumerate(sub_peaks):
            a,x0,sigma = sub_peaks[sub]
            fwhm = DM.fitting.sigma2FWHM(sigma)
            integral = DM.fitting.gaussIntegral(a,sigma)
            y_calc = DM.fitting.gauss(x[roi],a,x0,sigma)+bgr+bgr_lin
            
            peak_multi[f'{peak}_{sub}']['amplitude'].append(a)
            peak_multi[f'{peak}_{sub}']['position'].append(x0)
            peak_multi[f'{peak}_{sub}']['FWHM'].append(fwhm)
            peak_multi[f'{peak}_{sub}']['integral'].append(integral)
            peak_multi[f'{peak}_{sub}']['y_calc'].append(y_calc)

    # convert to numpy arrays
    for sub in sub_peaks:
        peak_multi[f'{peak}_{sub}'] = {key: np.array(peak_multi[f'{peak}_{sub}'][key]) for key in peak_multi[f'{peak}_{sub}']}

# compare the average observed peak to the average fitted peak
global_roi = min([min(peaks[peak]['roi']) for peak in peaks]), max([max(peaks[peak]['roi']) for peak in peaks])
roi = (x>global_roi[0]) & (x<global_roi[1])

plt.figure()
plt.title(DM.getScan_id(fname))
# plot the observed points
plt.plot(x[roi],np.mean(I,axis=0)[roi],'.',c='grey',label='average observed peak')
# plot the superposition of the fitted peaks
for peak in peaks:
    # create region of interest mask
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    plt.plot(x[roi],np.mean(peaks[peak]['I_calc'],axis=0),'-',c='grey')
# plot each fitted sub peak
for peak in peak_multi:
    # create region of interest mask
    roi = peak_multi[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    I_calc = peak_multi[peak]['y_calc']
    plt.plot(x[roi],np.nanmean(I_calc,axis=0),label=peak)
    
plt.ylabel('I [a.u.]')
if Q:
    plt.xlabel(r'Q [$\AA^{-1}$]')
else:
    plt.xlabel(r'2$\theta$ [$\deg$]')
plt.legend()


### Plot multi-peak fitting results

In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True)
fig.suptitle(DM.getScan_id(fname))
axs = axs.flatten()

keys = ['amplitude', 'position', 'FWHM', 'integral']
for peak in peak_multi:
    for i,key in enumerate(keys):
        axs[i].plot(t,peak_multi[peak][key],label=peak)
        axs[i].set_xlabel('Time (s)')
        axs[i].set_title(key)
# plot temperature if available
if type(T) != type(None):
    axs[-1].plot(t,T, label='Temperature')
    axs[-1].set_title('temperature')
else:
    axs[-1].plot(t,I0, label='I0')
    axs[-1].set_title('I0')

axs[-1].set_xlabel('Time (s)')
axs[2].legend()
fig.tight_layout()

### Plot heatmaps of observed and fitted peaks 

In [ ]:
fig, axs = plt.subplots(len(peaks), 3,sharey=True)
fig.suptitle(DM.getScan_id(fname))
axs = axs.flatten()

for i,peak in enumerate(peaks):
    ax0, ax1, ax2 = axs[i*3:i*3+3]
    ax1.sharex(ax0)
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    # obs-calc
    res = (I[:,roi]-peaks[peak]['I_calc'])/I[:,roi]
    
    ax0.set_title('observed')
    pcm = ax0.pcolormesh(x[roi],
                         t,
                         I[:,roi],
                         norm='log',
                        )
    vmin,vmax = pcm.get_clim()
    
    ax1.set_title('fitted')
    ax1.pcolormesh(x[roi],
                   t,
                   peaks[peak]['I_calc'],
                   vmin=vmin,
                   vmax=vmax,
                   norm='log',
                  )

    res_max = np.max(np.abs(res))
    ax2.set_title('residual')
    pcm = ax2.pcolormesh(x[roi],
                   t,
                   res,
                   vmin=-res_max,
                   vmax=res_max,
                   cmap='RdBu_r',
                  )
    fig.colorbar(pcm)